## Análise de Sono
Durante x dias eu realizei a coleta e registro dos dados do meu sono. 

Foi utilizado o Smartwatch Redmi GTS 2 Mini para coletar os dados.

Importação de bibliotecas

In [1]:
import pandas as pd

Transformando arquivo .csv em DataFrame

In [2]:
raw_data = pd.read_csv("../data/sleep_data.csv")

Visualização inicial 

In [3]:
raw_data

,Data,Dia da semana,Pontuação,Início,Fim,Duração,Regularidade,Sono profundo,REM,Tempo acordado,Vezes acordado
0,01/05,quinta-feira,72,22:39,06:43,07:58,90,00:39,01:42,6,3
1,02/05,sexta-feira,93,22:32,06:22,07:44,90,01:20,01:10,6,1
2,03/05,sábado,80,22:19,07:30,08:59,90,01:23,01:36,12,4
3,04/05,domingo,77,22:17,07:21,08:35,95,01:12,01:45,29,4
4,05/05,segunda-feira,85,22:08,05:31,07:08,89,01:12,01:38,15,2
...,...,...,...,...,...,...,...,...,...,...,...
68,08/07,terça-feira,78,22:33,07:20,08:39,71,01:03,01:58,8,3
69,09/07,quarta-feira,82,22:29,06:59,08:25,75,01:13,01:14,5,3
70,10/07,quinta-feira,76,00:34,08:30,07:52,74,00:58,01:30,4,2
71,11/07,sexta-feira,85,22:52,07:04,08:03,77,01:16,01:38,9,2


Verificação de Nulos

In [4]:
raw_data.isna().sum()

Data              0
Dia da semana     0
Pontuação         0
Início            0
Fim               0
Duração           0
Regularidade      0
Sono profundo     0
REM               0
Tempo acordado    0
Vezes acordado    0
dtype: int64

In [5]:
raw_data.describe().round(2)

,Pontuação,Regularidade,Tempo acordado,Vezes acordado
count,73.00,73.00,73.00,73.00
mean,79.71,80.05,11.18,2.08
std,6.77,6.15,9.77,1.32
min,61.00,69.00,0.00,0.00
25%,76.00,76.00,4.00,1.00
50%,80.00,79.00,9.00,2.00
75%,85.00,84.00,15.00,3.00
max,93.00,95.00,57.00,6.00


Padronizando nome das colunas para snake_case e remoção de acentos

In [6]:
raw_data = raw_data.rename(columns={
    "Data": "data", 
    "Dia da semana": "dia_semana", 
    "Pontuação": "pontuacao", 
    "Início": "inicio", 
    "Fim": "fim", 
    "Duração": "duracao",
    "Regularidade": "regularidade", 
    "Sono profundo": "sono_profundo", 
    "Tempo acordado": "tempo_acordado", 
    "Vezes acordado": "vezes_acordado"}
)

Conversão de dados de horas para minutos

In [7]:
raw_data["sono_profundo"] = raw_data["sono_profundo"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
raw_data["REM"] = raw_data["REM"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
raw_data["duracao"] = raw_data["duracao"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))

Cálculo de "Sono leve (min)" 

Subtraindo REM (min) e Sono profundo (min) da Duração (min)

In [8]:
raw_data["sono_leve"] = raw_data["duracao"] - raw_data["REM"] - raw_data["sono_profundo"]

Cálculo de porcentagem relativa 

REM - Sono profundo e Sono leve

In [9]:
raw_data["REM_perc"] = raw_data.apply(lambda x: round((x["REM"] / x["duracao"]) * 100, 2), axis=1)
raw_data["sono_profundo_perc"] = raw_data.apply(lambda x: round((x["sono_profundo"] / x["duracao"]) * 100, 2), axis=1)
raw_data["sono_leve_perc"] = raw_data.apply(lambda x: round((x["sono_leve"] / x["duracao"]) * 100, 2), axis=1)

Visualização do Data Frame após novas colunas criadas

In [10]:
raw_data.head()

,data,dia_semana,pontuacao,inicio,fim,duracao,regularidade,sono_profundo,REM,tempo_acordado,vezes_acordado,sono_leve,REM_perc,sono_profundo_perc,sono_leve_perc
0,01/05,quinta-feira,72,22:39,06:43,478,90,39,102,6,3,337,21.34,8.16,70.50
1,02/05,sexta-feira,93,22:32,06:22,464,90,80,70,6,1,314,15.09,17.24,67.67
2,03/05,sábado,80,22:19,07:30,539,90,83,96,12,4,360,17.81,15.40,66.79
3,04/05,domingo,77,22:17,07:21,515,95,72,105,29,4,338,20.39,13.98,65.63
4,05/05,segunda-feira,85,22:08,05:31,428,89,72,98,15,2,258,22.90,16.82,60.28


Conversão da coluna "Data" para formato datetime

In [11]:
from datetime import datetime

raw_data["data"] = pd.to_datetime(
    raw_data["data"] + f"/{datetime.now().year}", 
    format="%d/%m/%Y"
)


Formatação para remover as horas "00:00:00"

In [12]:
raw_data["data"] = pd.to_datetime(raw_data["data"]).dt.date

Após limpeza inicial, criação de novo Data Frame reorganizando as colunas iniciais

In [13]:
data = raw_data[["data", "dia_semana", "pontuacao","regularidade", 
                 "duracao", "sono_leve","sono_leve_perc", 
                 "sono_profundo", "sono_profundo_perc", "REM", "REM_perc", 
                 "tempo_acordado", "vezes_acordado"               
]]

In [14]:
data.head()

,data,dia_semana,pontuacao,regularidade,duracao,sono_leve,sono_leve_perc,sono_profundo,sono_profundo_perc,REM,REM_perc,tempo_acordado,vezes_acordado
0,2025-05-01,quinta-feira,72,90,478,337,70.50,39,8.16,102,21.34,6,3
1,2025-05-02,sexta-feira,93,90,464,314,67.67,80,17.24,70,15.09,6,1
2,2025-05-03,sábado,80,90,539,360,66.79,83,15.40,96,17.81,12,4
3,2025-05-04,domingo,77,95,515,338,65.63,72,13.98,105,20.39,29,4
4,2025-05-05,segunda-feira,85,89,428,258,60.28,72,16.82,98,22.90,15,2


In [15]:
data.to_csv("../data/cleaned.csv", index=False)